In [6]:
import os
import pandas as pd
import os
import sys
import argparse
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
from Bio import SeqIO
import re

wd = '/Users/allen/github/rujinlong/pc028e3-AvianViromeDB/analyses'
path_data = os.path.join(wd, "data/00-raw/d99-db_publish")
path_target = os.path.join(wd, "data/99-db_publish")
os.chdir(wd)

In [3]:
conn_raw = sqlite3.connect("pc028e3.sqlite")
conn_pub = sqlite3.connect("AvianViromeDB.sqlite")

In [12]:
def create_vOTU_fasta(fasta_genomes_input, fasta_genomes_output, conn):
    tmp_votu_id = "tmp_votu_id.csv"
    df_votu_id = pd.read_sql_query(f"SELECT contig_repid FROM contig_annotation_filtered_vOTU", conn)
    df_votu_id.to_csv(tmp_votu_id, header=False, index=False)

    # using seqkit to extract selected votus
    cmd = f"seqkit grep -f {tmp_votu_id} {fasta_genomes_input} -o {fasta_genomes_output}"
    subprocess.run(cmd, shell=True, check=True)

    # Remove temporary file
    os.remove(tmp_votu_id)
    return(df_votu_id)


def create_vOTU_protein(df_votu_ids, fasta_protin_input, fasta_protein_output):
    tmp_prot_id = "tmp_prot_ids.csv"
    tmp_protid_all = "tmp_protid_all.csv"
    df_votu_protein_ids = df_votu_ids.copy()
    df_votu_protein_ids['protein_id_prefix'] = df_votu_ids['contig_repid'] + '_'

    cmd = f"seqkit fx2tab -n {fasta_protin_input} | cut -d ' ' -f1 > {tmp_protid_all}"
    subprocess.run(cmd, shell=True, check=True)
    df_protid_all = pd.read_csv(tmp_protid_all, header=None, names=['protein_id'])
    df_protid_all['protein_id_prefix'] = df_protid_all['protein_id'].str.replace(r'_[0-9]*$', '_', regex=True)
    df_protid_sel = pd.merge(df_protid_all, df_votu_protein_ids, on='protein_id_prefix', how='inner')
    df_protid_sel[['protein_id']].to_csv(tmp_prot_id, header=False, index=False)

    cmd = f"seqkit grep -f {tmp_prot_id} {fasta_protin_input} -o {fasta_protein_output}"
    subprocess.run(cmd, shell=True, check=True)
    os.remove(tmp_prot_id)
    os.remove(tmp_protid_all)


def create_vOTU_annotation(conn_raw, conn_pub, output):
    df_vOTU_annotation = pd.read_sql_query(f"SELECT * FROM contig_annotation_filtered_vOTU", conn_raw)
    df_vOTU_annotation = df_vOTU_annotation.sort_values('vOTU_id')
    df_vOTU_annotation.to_csv(output, index=False, sep="\t")

    # Write the vOTU annotation data to the public database
    df_vOTU_annotation.to_sql('vOTU_annotation', conn_pub, if_exists='replace', index=False)
    return(df_vOTU_annotation)


def filter_by_vOTU(df, df_vOTU_annotation, conn_raw):
    df_ctgid2repid = pd.read_sql_query(f"SELECT contig_id, contig_repid, vOTU_id FROM contig_annotation", conn_raw)
    df_sel = pd.merge(df, df_ctgid2repid, on='contig_id', how='inner')
    df_sel = df_sel.drop('contig_id', axis=1)
    df_sel = df_sel[['contig_repid', 'vOTU_id'] + [col for col in df_sel.columns if col not in ['contig_repid', 'vOTU_id']]]
    df_sel = df_sel.sort_values('vOTU_id')
    df_sel_vOTU = pd.merge(df_sel, df_vOTU_annotation[["contig_repid"]], on='contig_repid', how='inner')
    return(df_sel_vOTU)


def create_vOTU_hosts(conn_raw, conn_pub, df_vOTU_annotation, output):
    df_prokaryotic_hosts = pd.read_sql_query(f"SELECT * FROM host_filtered", conn_raw)
    df_prokyriotic_hosts_sel_vOTU = filter_by_vOTU(df_prokaryotic_hosts, df_vOTU_annotation, conn_raw)
    # output
    df_prokyriotic_hosts_sel_vOTU.to_csv(output, index=False, sep="\t")
    df_prokyriotic_hosts_sel_vOTU.to_sql('virus_prokyriotic_hosts', conn_pub, if_exists='replace', index=False)
    return(df_prokyriotic_hosts_sel_vOTU)


def create_vOTU_amg(conn_raw, conn_pub, df_vOTU_annotation, output, software):
    """
    software: 'dramv' or 'vibrant'
    """
    df_amg = pd.read_sql_query(f"SELECT * FROM amg_{software}_filtered", conn_raw)
    df_amg_sel_vOTU = filter_by_vOTU(df_amg, df_vOTU_annotation, conn_raw)
    # output
    df_amg_sel_vOTU.to_csv(output, index=False, sep="\t")
    df_amg_sel_vOTU.to_sql(f"amg_{software}", conn_pub, if_exists='replace', index=False)
    return(df_amg_sel_vOTU)


In [14]:
fasta_genomes = os.path.join(path_data, "ccflt.fasta.gz")
fasta_protin_input = os.path.join(path_data, "proteins.faa")
fasta_protein_output = os.path.join(path_target, "vOTU_proteins.faa.gz")

df_votu_ids = create_vOTU_fasta(fasta_genomes, f"{path_target}/vOTUs.fasta.gz", conn_raw)
df_vOTU_annotation = create_vOTU_annotation(conn_raw, conn_pub, f"{path_target}/vOTU_annotation.tsv.gz")
df_prokyriotic_hosts = create_vOTU_hosts(conn_raw, conn_pub, df_vOTU_annotation, f"{path_target}/vOTU_prokyriotic_hosts.tsv.gz")

df_amg_dramv = create_vOTU_amg(conn_raw, conn_pub, df_vOTU_annotation, f"{path_target}/AMG_DRAMv.tsv.gz", "dramv")
df_amg_vibrant = create_vOTU_amg(conn_raw, conn_pub, df_vOTU_annotation, f"{path_target}/AMG_VIBRANT.tsv.gz", "vibrant")
os.chdir(path_target)
files_tar = ["AMG_DRAMv.tsv.gz", "AMG_VIBRANT.tsv.gz"]
command = ['tar', '-czf', f'{path_target}/gene_annotations.tar.gz'] + files_tar
subprocess.run(command, check=True, text=True)

# Remove the files after creating the tar archive
for file in files_tar:
    os.remove(file)

os.chdir(wd)

create_vOTU_protein(df_votu_ids, fasta_protin_input, fasta_protein_output)

[INFO] 61608 patterns loaded from file
[INFO] 2322919 patterns loaded from file


In [5]:
# read all contig_repid column and store in a dataframe
# Read all contig_repid columns from all tables in the database
cursor = conn_raw.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Create an empty list to store dataframes
dfs = []

# Iterate through each table and read contig_repid column if it exists
for table in tables:
    table_name = table[0]
    try:
        # Check if contig_repid column exists in the table
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = [col[1] for col in cursor.fetchall()]
        
        if 'contig_repid' in columns:
            # Read the contig_repid column
            df = pd.read_sql_query(f"SELECT contig_repid FROM {table_name}", conn_raw)
            df['source_table'] = table_name  # Add source table name
            dfs.append(df)
    except:
        continue

# Combine all dataframes
if dfs:
    contig_repid_df = pd.concat(dfs, ignore_index=True)
    print(f"Total number of contig_repid entries: {len(contig_repid_df)}")
    print("\nFirst few rows:")
    print(contig_repid_df.head())
else:
    print("No contig_repid columns found in any tables")



Total number of contig_repid entries: 428735

First few rows:
   contig_repid                source_table
0  contig002642  contig_annotation_filtered
1  contig000076  contig_annotation_filtered
2  contig000120  contig_annotation_filtered
3  contig000144  contig_annotation_filtered
4  contig000193  contig_annotation_filtered


In [4]:
# list all tables in conn_raw
cursor = conn_raw.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    print(table[0])


contig_annotation_filtered_vOTU_infomap
amg_dramv_filtered
amg_vibrant_filtered
host_filtered
vcontact_filtered
contig_annotation_filtered
sample_metadata_filtered
contig_annotation_filtered_vOTU
anno_prot_eggnog
anno_prot_dramv
vcontact
contig_annotation
host
sample_metadata
amg_dramv
amg_vibrant


In [ ]:
# 1. Create vOTU fasta file

# read sqlite database AvianViromeDB.db

conn_raw = sqlite3.connect("pc028e3.sqlite")
conn = sqlite3.connect("AvianViromeDB.sqlite")

# read table vOTUs
vOTUs = pd.read_sql_query("SELECT * FROM vOTUs", conn)

# create fasta file
genome_records = SeqIO.parse(vOTUs['vOTU_seq'], "fasta")





# 2. Create vOTU annotation file

# 3. Create AMG/ARG annotation file

# 4. Create host prediction file



In [38]:
def read_md5(fin, new_col_name="assembly_id"):
    df = pl.read_csv(fin, separator="\t")
    df = df.rename({"Header": new_col_name})
    return df.to_pandas()

df_qry = read_md5("ccflt.tsv", "contig_id")
df_o1 = read_md5("o1.tsv", "assembly_id")
df_o2 = read_md5("o2.tsv", "assembly_id")
df_o3 = read_md5("o3.tsv", "assembly_id")
df_o4 = read_md5("o4.tsv", "assembly_id")
df_imeta = read_md5("imeta.tsv", "assembly_id")

In [39]:
# row bind df_o1, df_o2, df_o3, df_imeta, df_o4
df_target = pd.concat([df_o1, df_o2, df_o3, df_o4, df_imeta], ignore_index=True)
df_target.shape[0]

10452355

In [40]:
df_target.head()

,assembly_id,MD5
0,SRR17030618__k141_136590_flag=1_multi=5.0000_l...,6896701004b080aa9c71e434dff0aee5
1,SRR17030618__k141_375563_flag=1_multi=4.8475_l...,a16091528787c01d454e470a3c057c6b
2,SRR17030618__k141_273129_flag=1_multi=15.3704_...,7322b194074a2ef8f5d7f2a239b6f4f3
3,SRR17030618__k141_52_flag=0_multi=10.0000_len=...,926ddc3d00e1e8f5093c867487acc9e2
4,SRR17030618__k141_136592_flag=1_multi=5.0000_l...,acf8553ec37799e95bf397f24125881c


In [41]:
df_qry.shape[0]

301456

In [42]:
# only use the non-duplicated MD5 in df_qry. Remove all the duplicated rows in df_qry and only keep the first row
df_qry_non_dup = df_qry[~df_qry.duplicated("MD5", keep="first")]
df_qry_non_dup.shape[0]

297018

In [43]:
df_rst = df_qry.merge(df_target, on="MD5", how="inner")
df_rst.head()

,contig_id,MD5,assembly_id
0,contig000001,fe55d48fd62287a967f0d52b23423b2e,ma2024imeta__ANCeC1__1885
1,contig000002,5eebad83a264a6a55dab14a0aa452003,ma2024imeta__ANCeC1__2100
2,contig000003,63bfdd264bf11f4c471cc5466a020071,ma2024imeta__ANCeC1__4767
3,contig000004,33c34c41abbaa151a6b3fdee2ce28cf1,ma2024imeta__ANCeC1__4881
4,contig000005,f23673c7e6972750aa462c3ac0754c1e,ma2024imeta__ANCeC1__5050


In [45]:
df_rst.shape[0]

309877

In [46]:
rst = df_rst[["contig_id", "assembly_id"]]
# remove duplicated contig_id
rst = rst[~rst.duplicated("contig_id", keep="first")]

In [48]:
rst.shape[0]

290876

In [49]:
# save rst to csv 
rst.to_csv("map_contig2asb.tsv", index=False, header=False, sep="\t")

In [36]:
df_empty = pd.read_csv("df_asb_empty.tsv", sep="\t")

In [22]:
df_empty

,contig_repid,contig_id,vOTU_id,assembly_id,sample_id,contig_length,checkv_provirus,checkv_quality_score,checkv_completeness,checkv_contamination,...,virsorter2_group,virsorter2_category,vibrant_quality_score,vibrant_type,vc_genus,vc_classified,vc_id,vc_with_ref,vc_novel_genus,lifestyle
0,contig000027,contig136944,vOTU000014,NaN,NaN,13533,No,4,28.62,0.0,...,dsDNAphage,2.0,4.0,lytic,novel_genus_41_of_novel_subfamily_0_of_novel_f...,classified,VC57759,False,True,virulent
1,contig000027,contig133866,vOTU000014,NaN,NaN,13533,No,4,28.62,0.0,...,dsDNAphage,2.0,4.0,lytic,novel_genus_41_of_novel_subfamily_0_of_novel_f...,classified,VC57759,False,True,virulent
2,contig000027,contig133729,vOTU000014,NaN,NaN,13533,No,4,28.62,0.0,...,dsDNAphage,2.0,4.0,lytic,novel_genus_41_of_novel_subfamily_0_of_novel_f...,classified,VC57759,False,True,virulent
3,contig000027,contig133720,vOTU000014,NaN,NaN,13533,No,4,28.62,0.0,...,dsDNAphage,2.0,4.0,lytic,novel_genus_41_of_novel_subfamily_0_of_novel_f...,classified,VC57759,False,True,virulent
4,contig000027,contig133678,vOTU000014,NaN,NaN,13533,No,4,28.62,0.0,...,dsDNAphage,2.0,4.0,lytic,novel_genus_41_of_novel_subfamily_0_of_novel_f...,classified,VC57759,False,True,virulent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20679,contig301345,contig301345,vOTU139137,NaN,NaN,32412,No,3,79.14,0.0,...,dsDNAphage,1.0,3.0,lysogenic,novel_genus_3_of_novel_subfamily_0_of_novel_fa...,classified,VC36605,False,True,temperate
20680,contig301346,contig301346,vOTU139138,NaN,NaN,66606,No,3,60.03,0.0,...,dsDNAphage,1.0,4.0,lysogenic,novel_genus_6_of_novel_subfamily_0_of_novel_fa...,classified,VC14789,False,True,temperate
20681,contig301347,contig301347,vOTU139139,NaN,NaN,26221,No,4,23.57,0.0,...,dsDNAphage,1.0,4.0,lytic,novel_genus_15_of_novel_subfamily_0_of_novel_f...,classified,VC14826,False,True,virulent
20682,contig301348,contig301348,vOTU139140,NaN,NaN,21121,No,4,40.51,0.0,...,dsDNAphage,2.0,4.0,lytic,novel_genus_1_of_novel_subfamily_0_of_novel_fa...,classified,VC10477,False,True,virulent


In [21]:
df_o1.shape[0], df_o2.shape[0], df_o3.shape[0], df_imeta.shape[0]

(7196266, 13307088, 271875, 1996937)

In [18]:
def filter_df_by_length(df_qry, df_target):
    df_target = df_target.merge(df_qry[["Length"]], on="Length", how="inner")
    # only show Header column in the output
    return df_target[["Header"]]

rst_o1 = filter_df_by_length(df_qry, df_o1)
rst_o2 = filter_df_by_length(df_qry, df_o2)
rst_o3 = filter_df_by_length(df_qry, df_o3)
rst_imeta = filter_df_by_length(df_qry, df_imeta)

In [20]:
rst_o1.shape[0], rst_o2.shape[0], rst_o3.shape[0], rst_imeta.shape[0]

(1981512, 13290562, 269493, 1912864)